In [8]:
import torch
from torch import nn
import tiktoken
from torch.utils.data import Dataset, DataLoader

#### Embedding（将token id嵌入）

#### 嵌入层只是独热编码和矩阵乘法方法的一种更高效的实现，因此它可以被视为一个能够通过反向传播进行优化的神经网络层。

#### 绝对位置嵌入：词向量 + 位置向量
#### 相对位置嵌入：加入依赖位置差

In [9]:
# Dataset
class GPTDataset(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        super().__init__()
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt)
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1:i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [10]:
vocab_size = 50257
output_dim = 256
# 构建embedding层
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
tokenizer = tiktoken.get_encoding("gpt2")
# DataLoader
def GPTDataLoader(txt, batch_size, max_length, stride, shuffle = True, 
                  drop_last = True, num_workers = 0, tokenizer = tokenizer):
    dataset = GPTDataset(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        drop_last=drop_last,
        num_workers=num_workers,
        shuffle=shuffle
    )
    return dataloader
with open("the-verdict.txt", "r", encoding='utf-8') as f:
    raw_text = f.read()
dataloader = GPTDataLoader(raw_text, batch_size=8, max_length=4,
stride=4, shuffle=False)
data_iter = iter(dataloader)
inputs, target = next(data_iter)
print("Token IDs:\n", inputs) 
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [11]:
token_embeddings = token_embedding_layer(inputs) 
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [12]:
# 得到绝对位置编码
context_length = 4
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length)) 
print(pos_embeddings.shape)

torch.Size([4, 256])


In [13]:
# 最终得到的embedding向量
input_embedding = token_embeddings + pos_embeddings

### One-Hot编码

#### one-hot编码，用一个向量表示一个物体，Embedding就是one-hot编码的压缩变换

### Word2Vec

#### Word2Vec是训练Embedding 的一种方法
#### 直接吧词转换为向量
#### 包括CBOW和Skip-Gram两种方法

### FastText

#### 与Word2Vec类似，都是根据上下文预测，但是每个词由多个子词向量相加而成

### Glove

### 上面两种是居于局部上下文进行预测，Glove是居于全局的统计信息预测
### 🧩 1. 构建共现矩阵

在整个语料库中，统计每个词与上下文词出现的次数：

- \( X_{ij} \)：单词 \( j \) 在单词 \( i \) 的上下文中出现的次数  
- \( X_i = \sum_k X_{ik} \)：单词 \( i \) 的上下文总出现次数  

于是可以得到共现概率：

\[
P_{ij} = \frac{X_{ij}}{X_i}
\]

表示「在词 \( i \) 的上下文中看到词 \( j \)」的概率。

---

### 🔍 2. 共现比值反映语义关系

若两个词 \( i, j \) 与一个上下文词 \( k \) 的共现概率相似，说明它们语义接近。

例如：

- “ice” 与 “solid” 共现较高  
- “steam” 与 “gas” 共现较高  
- “ice” 与 “gas” 共现较低  

这种语义关系可以用 **共现概率比值** 表达：

\[
\frac{P_{ik}}{P_{jk}}
\]

这个比值捕捉了「词与词之间的语义差异」。

---

### ⚖️ 3. 模型目标

我们希望找到词向量 \( w_i \) 和上下文向量 \( \tilde{w}_j \)，  
使得它们的关系能近似地表示这个概率比值：

\[
f(w_i, w_j, \tilde{w}_k) = \frac{P_{ik}}{P_{jk}}
\]

通过一系列推导，GloVe 得出一个更简单的线性方程：

\[
w_i^T \tilde{w}_j + b_i + \tilde{b}_j = \log(X_{ij})
\]

即：

> 两个词向量的点积 + 偏置 ≈ 它们共现次数的对数

---

### 🎯 4. 最终优化目标（损失函数）

GloVe 使用加权平方误差最小化整个语料的误差：

\[
J = \sum_{i,j=1}^{V} f(X_{ij}) \, (w_i^T \tilde{w}_j + b_i + \tilde{b}_j - \log X_{ij})^2
\]

其中：

- \( f(X_{ij}) \) 是权重函数，抑制极端频率的影响：
  \[
  f(x) = 
  \begin{cases}
  (x / x_{\text{max}})^{\alpha}, & \text{if } x < x_{\text{max}} \\
  1, & \text{otherwise}
  \end{cases}
  \]

---

### 💡 5. 向量语义示例

GloVe 学得的词向量具有语义运算特性：

\[
\text{vec}("king") - \text{vec}("man") + \text{vec}("woman") \approx \text{vec}("queen")
\]
